# Render your design to Gmsh
This notebook demonstrates the new open-source rendering capabilities of Qiskit Metal using Gmsh for generating meshes for the design of interest. The tutorial has the following steps:

## Contents

### 1. Creating a Transmon coupled to a resonator in Qiskit Metal

### 2. Rendering your design using `QGmshRenderer`
- Rendering the wireframe of whole design
- Rendering the components selectively
- Defining open and short pins
- Skipping junctions

### 3. Meshing your design using `QGmshRenderer`
- Applying a basic mesh to your design
- Customizing the mesh using initial mesh size parameters
- Using the ***Intelli-mesh*** feature
- **Advanced:** defining your own custom meshing function

### 4. Exporting your mesh to a file
- Use `QGmshRenderer` to export the generated mesh to a file

# Some necessary imports

In [ ]:
%load_ext autoreload
%autoreload 2

from qiskit_metal import MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

In [ ]:
%metal_heading 1. Creating a Transmon coupled to a resonator in Qiskit Metal

In [ ]:
design = designs.MultiPlanar({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [ ]:
design.delete_all_components()

# Making the qubit
q_opts = dict(
    pos_x = "2mm",
    pos_y = "2mm",
    connection_pads = dict(
        readout = dict()
    ),
    chip="main",
    layer=1
)

q1 = TransmonPocket(design, "Q1", options=q_opts)

# Making the open to ground
otg_opts = dict(
    pos_x = "4mm",
    pos_y = "2mm",
    chip="main",
    layer=1
)

otg1 = OpenToGround(design, "otg1", options=otg_opts)

# Making the meandered resonator
meander_opts1 = dict(
    pin_inputs = dict(
        start_pin = dict(component = "Q1", pin = "readout"),
        end_pin = dict(component = "otg1", pin = "open")
    ),
    lead = dict(
        start_straight = "100um"
    ),
    fillet = "99.99um",
    asymmetry = "100um",
    chip="main",
    layer=1
)

mtl1 = RouteMeander(design, "mtl1", options=meander_opts1)

gui.rebuild()
gui.autoscale()

In [ ]:
%metal_heading 2. Rendering your design using `QGmshRenderer`

In [ ]:
from qiskit_metal.renderers.renderer_gmsh.gmsh_renderer import QGmshRenderer

# Instantiate QGmshRenderer
gmsh_renderer = QGmshRenderer(design)

### Rendering the wireframe of whole design

In [ ]:
# Render the design in Gmsh (wireframe only)
gmsh_renderer.render_design(mesh_geoms=False)

# Launch Gmsh GUI to verify
gmsh_renderer.launch_gui()

### Rendering the components selectively

In [ ]:
# Rendering the 'Q1' component
gmsh_renderer.render_design(selection=['Q1'], mesh_geoms=False)
gmsh_renderer.launch_gui()

In [ ]:
# Rendering the 'mtl1' component
gmsh_renderer.render_design(selection=['mtl1'], mesh_geoms=False)
gmsh_renderer.launch_gui()

### Defining `open_pins`

In [ ]:
# Rendering the 'Q1' component with open_pins
gmsh_renderer.render_design(selection=['Q1'],
                            open_pins=[('Q1', 'readout')],
                            mesh_geoms=False)
gmsh_renderer.launch_gui()

### Skipping the junction

In [ ]:
# Rendering the 'Q1' component without the junction
gmsh_renderer.render_design(selection=['Q1'], mesh_geoms=False, skip_junctions=True)
gmsh_renderer.launch_gui()

In [ ]:
%metal_heading 3. Meshing your design using `QGmshRenderer`

### Applying a basic mesh to your design
Run the code below, then in the Gmsh app window do the following to see the mesh on the chip
1. In the menu bar on top, go to `Tools > Visibility`
1. In the visbility settings window, click on the drop-down and select `Physical Groups`
1. Press Ctrl-key (or Command-key on MacOS) and select everything except the ones having `vacuum_box...` in their name.
1. Click on Apply, and close the visibility settings window.
1. In the main Gmsh window, right click and go to `Mesh Visibility`, and select the option `2D element faces`
1. This will make your chip's mesh visible.

In [ ]:
# meshing the whole design
gmsh_renderer.render_design(mesh_geoms=False)
gmsh_renderer.add_mesh(intelli_mesh=False)
gmsh_renderer.launch_gui()

### Customizing the mesh using initial mesh size parameters
As we can observe in the output from previous cell, the mehs nodes are very big and for small components like the junction or the CPW structures, it might not be a good approximation by having such big mesh elements. One way to make a finer mesh is to set the initial mesh size constraints like shown below:

In [ ]:
# Setting initial mesh size constraints (might take a while to run!!)
# DISCLAIMER: this might also crash if your system has less than 8GB of memory
gmsh_renderer.options.mesh.max_size = "20um"
gmsh_renderer.options.mesh.min_size = "5um"
gmsh_renderer.render_design(mesh_geoms=False)
gmsh_renderer.add_mesh(intelli_mesh=False)
gmsh_renderer.launch_gui()

### Using the Intelli-mesh feature
As we observe from the previous cell, just setting the intial mesh size constraints to a small value is very computationally intensive and requires a lot of memory. Hence, in a practical scenario we try to increase the mesh density in the regions which are of critical importance and decrease the mesh density where it does not matter much.

In `QGmshRenderer`, we have a semi-automatic way of meshing the design taking the assumption that the electric and magnetic fields would always be concentrated at the edges of metals and at the metal-dielectric interfaces. This whole thing is wrapped up neatly in a feature that we call **Intelli-mesh**!

In [ ]:
# Intelli-mesh is ON by default and will work
# automatically when calling the render_design() method
gmsh_renderer.options.mesh.max_size = "70um"
gmsh_renderer.options.mesh.min_size = "5um"
gmsh_renderer.render_design()
gmsh_renderer.launch_gui()

### **Advanced:** defining your own custom meshing function
This is only for advanced users who are aware of how Gmsh works and how to specify the mesh size fields for Gmsh.

In [ ]:
# Define a function to specify custom mesh density field across your design
def my_custom_meshing_function():
    
    # Selectively increase mesh density for the following QGeometries
    q1_top_pad = gmsh_renderer.polys_dict[1]["Q1_pad_top"][0]
    q1_readout_wire = gmsh_renderer.paths_dict[1]["Q1_readout_wire"][0]
    
    # Extract surfaces from Gmsh volume tags
    volumes_list = [q1_top_pad, q1_readout_wire]
    surfaces = []
    for vol in volumes_list:
        surfaces += [tag for tag in gmsh_renderer.modeler.occ.getSurfaceLoops(vol)[1][0]]
    
    # Extract min and max mehs size constraints from the renderer
    min_size = gmsh_renderer.parse_units_gmsh(gmsh_renderer.options.mesh.min_size)
    max_size = gmsh_renderer.parse_units_gmsh(gmsh_renderer.options.mesh.max_size)
    
    # Define Gmsh fields
    # Field: Distance
    df = gmsh_renderer.modeler.mesh.field.add("Distance")
    gmsh_renderer.modeler.mesh.field.setNumbers(df, "SurfacesList", surfaces)
    gmsh_renderer.modeler.mesh.field.setNumber(df, "NumPointsPerCurve", 100)
    
    # Field: Threshold
    tf = gmsh_renderer.modeler.mesh.field.add("Threshold")
    gmsh_renderer.modeler.mesh.field.setNumber(tf, "DistMin", 0.01)
    gmsh_renderer.modeler.mesh.field.setNumber(tf, "DistMax", 0.07)
    gmsh_renderer.modeler.mesh.field.setNumber(tf, "Sigmoid", 1)
    gmsh_renderer.modeler.mesh.field.setNumber(tf, "InField", df)
    gmsh_renderer.modeler.mesh.field.setNumber(tf, "SizeMin", min_size)
    gmsh_renderer.modeler.mesh.field.setNumber(tf, "SizeMax", max_size)
    
    # Set as background mesh to generate the mesh
    gmsh_renderer.modeler.mesh.field.setAsBackgroundMesh(tf)

In [ ]:
# define initial mesh sizes
gmsh_renderer.options.mesh.max_size = "70um"
gmsh_renderer.options.mesh.min_size = "5um"

# render design wireframe
gmsh_renderer.render_design(selection=['Q1'], mesh_geoms=False)

# Pass your custom mehsing function to add_mesh()
gmsh_renderer.add_mesh(intelli_mesh=True, custom_mesh_fn=my_custom_meshing_function)
gmsh_renderer.launch_gui()

In [ ]:
%metal_heading 4. Exporting your mesh to a file

### Use `QGmshRenderer` to export the generated mesh to a file

In [ ]:
# Export the mesh to a file
gmsh_renderer.export_mesh("test.msh")

In [ ]:
# Close and destroy the renderer object
gmsh_renderer.close()

In [ ]:
gui.main_window.close()